In [10]:

# Install a Drive FUSE wrapper.# Insta 
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

gpg: keybox '/tmp/tmp0nsmqllh/pubring.gpg' created
gpg: /tmp/tmp0nsmqllh/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [12]:

# Generate creds for the Drive FUSE library.# Gener 
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
# Work around misordering of STREAM and STDIN in Jupyter.
# https://github.com/jupyter/notebook/issues/3159
prompt = !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass(prompt[0] + '\n\nEnter verification code: ')
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

IndexError: ignored

In [4]:
!mkdir -p drive
!google-drive-ocamlfuse drive

print ('Files in Drive:')
!ls drive/


Files in Drive:
1115-resume-powerpoint-template-(cv-template).pdf
4_6030726325214380780.odt
AP-Workshop-Session2.odt
Back transliteration.ods
Colab Notebooks
created.txt
cross language survey.odt
fra.txt
google_analogy2.txt
google_analogy.txt
Homeworks-G2.ods
IMG_20141110_092043~2.jpg
Marks.ods
myself.jpg
NER & Word Clustering.ods
New Doc 2.pdf
Open source Tookits or Frameworks or ....ods
Paper revision.odt
para2vec.odt
Persian corpus information.ods
presentation
resized_myself.jpg
resized_starrynight.jpg
simlex999-translated.xlsx
simlex999-translated.xlsx.ods
simlex-final.ods
SNLP_PRESENTATION.pdf
SNLP_PRESENTATION.pptx
SourceCodes
starrynight.jpg
survey.odt
Theoretical Interview.pdf
transliteration.odt
university position.ods
Untitled document.odt
Untitled document.odt (75ca89bf)
Untitled presentation.pdf
Untitled presentation.pdf (58a532f6)
Useful things to know.ods
Word embedding.ods
آموزش ورد۲وک.odt
برچسب سوالات.odt
پرسشنامه ۲ (Responses).ods
پرسشنامه ۲.zip
پرسشنامه.zip
جنسیت (Res

In [0]:
# -*- coding: utf-8 -*-
#
# Copyright 2018 Amir Hadifar. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

import os

import numpy as np
import scipy.misc
from PIL import Image, ImageOps
from six.moves import urllib


def download(download_link, file_name, expected_bytes):
    """ Download the pretrained VGG-19 model if it's not already downloaded """
    if os.path.exists(file_name):
        print("VGG-19 pre-trained model is ready")
        return
    print("Downloading the VGG pre-trained model. This might take a while ...")
    file_name, _ = urllib.request.urlretrieve(download_link, file_name)
    file_stat = os.stat(file_name)
    if file_stat.st_size == expected_bytes:
        print('Successfully downloaded VGG-19 pre-trained model', file_name)
    else:
        raise Exception('File ' + file_name +
                        ' might be corrupted. You should try downloading it with a browser.')


def get_resized_image(img_path, width, height, save=True):
    image = Image.open(img_path)
    # PIL is column major so you have to swap the places of width and height
    image = ImageOps.fit(image, (width, height), Image.ANTIALIAS)
    if save:
        image_dirs = img_path.split('/')
        image_dirs[-1] = 'resized_' + image_dirs[-1]
        out_path = '/'.join(image_dirs)
        if not os.path.exists(out_path):
            image.save(out_path)
    image = np.asarray(image, np.float32)
    return np.expand_dims(image, 0)


def generate_noise_image(content_image, width, height, noise_ratio=0.6):
    noise_image = np.random.uniform(-20, 20, (1, height, width, 3)).astype(np.float32)
    return noise_image * noise_ratio + content_image * (1 - noise_ratio)


def save_image(path, image):
    image = image[0]
    image = np.clip(image, 0, 255).astype('uint8')
    scipy.misc.imsave(path, image)


def safe_mkdir(path):
    """ Create a directory if there isn't one already. """
    try:
        os.mkdir(path)
    except OSError:
        pass


In [0]:
# -*- coding: utf-8 -*-
#
# Copyright 2018 Amir Hadifar. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

import numpy as np
import scipy.io
import tensorflow as tf

# VGG-19 parameters file
VGG_DOWNLOAD_LINK = 'http://www.vlfeat.org/matconvnet/models/imagenet-vgg-verydeep-19.mat'
VGG_FILENAME = 'imagenet-vgg-verydeep-19.mat'
EXPECTED_BYTES = 534904783


class VGG(object):
    def __init__(self, input_img):
        download(VGG_DOWNLOAD_LINK, VGG_FILENAME, EXPECTED_BYTES)
        self.vgg_layers = scipy.io.loadmat(VGG_FILENAME)['layers']
        self.input_img = input_img
        self.mean_pixels = np.array([123.68, 116.779, 103.939]).reshape((1, 1, 1, 3))

    def _weights(self, layer_idx, expected_layer_name):
        """ Return the weights and biases at layer_idx already trained by VGG
        """
        W = self.vgg_layers[0][layer_idx][0][0][2][0][0]
        b = self.vgg_layers[0][layer_idx][0][0][2][0][1]
        layer_name = self.vgg_layers[0][layer_idx][0][0][0][0]
        assert layer_name == expected_layer_name
        return W, b.reshape(b.size)

    def conv2d_relu(self, prev_layer, layer_idx, layer_name):
        """ Create a convolution layer with RELU using the weights and
        biases extracted from the VGG model at 'layer_idx'. You should use
        the function _weights() defined above to extract weights and biases.
        _weights() returns numpy arrays, so you have to convert them to TF tensors.
        Don't forget to apply relu to the output from the convolution.
        Inputs:
            prev_layer: the output tensor from the previous layer
            layer_idx: the index to current layer in vgg_layers
            layer_name: the string that is the name of the current layer.
                        It's used to specify variable_scope.
        Hint for choosing strides size:
            for small images, you probably don't want to skip any pixel
        """
        W, b = self._weights(layer_idx, layer_name)
        W, b = tf.convert_to_tensor(W), tf.convert_to_tensor(b)
        conv2d = tf.nn.conv2d(prev_layer, W, strides=[1, 1, 1, 1], padding="SAME")
        out = tf.nn.relu(tf.nn.bias_add(conv2d, b))
        setattr(self, layer_name, out)

    def avgpool(self, prev_layer, layer_name):
        """ Create the average pooling layer. The paper suggests that
        average pooling works better than max pooling.

        Input:
            prev_layer: the output tensor from the previous layer
            layer_name: the string that you want to name the layer.
                        It's used to specify variable_scope.
        Hint for choosing strides and kszie: choose what you feel appropriate
        """
        out = tf.nn.avg_pool(prev_layer, [1, 2, 2, 1], [1, 2, 2, 1], padding="SAME")
        setattr(self, layer_name, out)

    def load(self):
        self.conv2d_relu(self.input_img, 0, 'conv1_1')
        self.conv2d_relu(self.conv1_1, 2, 'conv1_2')
        self.avgpool(self.conv1_2, 'avgpool1')
        self.conv2d_relu(self.avgpool1, 5, 'conv2_1')
        self.conv2d_relu(self.conv2_1, 7, 'conv2_2')
        self.avgpool(self.conv2_2, 'avgpool2')
        self.conv2d_relu(self.avgpool2, 10, 'conv3_1')
        self.conv2d_relu(self.conv3_1, 12, 'conv3_2')
        self.conv2d_relu(self.conv3_2, 14, 'conv3_3')
        self.conv2d_relu(self.conv3_3, 16, 'conv3_4')
        self.avgpool(self.conv3_4, 'avgpool3')
        self.conv2d_relu(self.avgpool3, 19, 'conv4_1')
        self.conv2d_relu(self.conv4_1, 21, 'conv4_2')
        self.conv2d_relu(self.conv4_2, 23, 'conv4_3')
        self.conv2d_relu(self.conv4_3, 25, 'conv4_4')
        self.avgpool(self.conv4_4, 'avgpool4')
        self.conv2d_relu(self.avgpool4, 28, 'conv5_1')
        self.conv2d_relu(self.conv5_1, 30, 'conv5_2')
        self.conv2d_relu(self.conv5_2, 32, 'conv5_3')
        self.conv2d_relu(self.conv5_3, 34, 'conv5_4')
        self.avgpool(self.conv5_4, 'avgpool5')


In [30]:
!mkdir drive/outputs
!rm -r graphs
!rm -r checkpoints
!rm -r outputs
!ls

mkdir: cannot create directory ‘drive/outputs’: File exists
adc.json  datalab  drive  imagenet-vgg-verydeep-19.mat	sample_data


In [29]:
# -*- coding: utf-8 -*-
#
# Copyright 2018 Amir Hadifar. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import time

import numpy as np
import tensorflow as tf
tf.reset_default_graph()

def setup():
    safe_mkdir('checkpoints')
    safe_mkdir('outputs')


class StyleTransfer(object):
    def __init__(self, content_img, style_img, img_width, img_height):
        """
        img_width and img_height are the dimensions we expect from the generated image.
        We will resize input content image and input style image to match this dimension.
        Feel free to alter any hyperparameter here and see how it affects your training.
        """
        self.img_width = img_width
        self.img_height = img_height
        self.content_img = get_resized_image(content_img, img_width, img_height)
        self.style_img = get_resized_image(style_img, img_width, img_height)
        self.initial_img = generate_noise_image(self.content_img, img_width, img_height)

        self.content_layer = 'conv4_2'
        self.style_layers = ['conv1_1', 'conv2_1', 'conv3_1', 'conv4_1', 'conv5_1']
        # content_w, style_w: corresponding weights for content loss and style loss
        self.content_w = 0.01
        self.style_w = 1
        # style_layer_w: weights for different style layers. deep layers have more weights
        self.style_layer_w = [0.5, 1.0, 1.5, 3.0, 4.0]
        self.gstep = tf.get_variable(name='global_step', initializer=0, trainable=False)  # global step
        self.lr = 1.0

    def create_input(self):
        """
        We will use one input_img as a placeholder for the content image,
        style image, and generated image, because:
            1. they have the same dimension
            2. we have to extract the same set of features from them
        We use a variable instead of a placeholder because we're, at the same time,
        training the generated image to get the desirable result.
        Note: image height corresponds to number of rows, not columns.
        """
        with tf.variable_scope('input'):
            self.input_img = tf.get_variable('in_img',
                                             shape=([1, self.img_height, self.img_width, 3]),
                                             dtype=tf.float32,
                                             initializer=tf.zeros_initializer())

    def load_vgg(self):
        """
        Load the saved model parameters of VGG-19, using the input_img
        as the input to compute the output at each layer of vgg.
        During training, VGG-19 mean-centered all images and found the mean pixels
        to be [123.68, 116.779, 103.939] along RGB dimensions. We have to subtract
        this mean from our images.
        """
        self.vgg = VGG(self.input_img)
        self.vgg.load()
        self.content_img -= self.vgg.mean_pixels
        self.style_img -= self.vgg.mean_pixels

    def _content_loss(self, P, F):
        """ Calculate the loss between the feature representation of the
        content image and the generated image.

        Inputs:
            P: content representation of the content image
            F: content representation of the generated image
            Read the assignment handout for more details
            Note: Don't use the coefficient 0.5 as defined in the paper.
            Use the coefficient defined in the assignment handout.
        """
        self.content_loss = tf.reduce_sum(tf.square(F - P)) / (4 * P.size)

    def _gram_matrix(self, F, N, M):
        """ Create and return the gram matrix for tensor F
            Hint: you'll first have to reshape F
        """
        # N third dim of feature map
        # M product of first two dim of feature map
        # F feature map
        F = tf.reshape(F, [M, N])
        return tf.matmul(F, F, transpose_a=True)

    def _single_style_loss(self, a, g):
        """ Calculate the style loss at a certain layer
        Inputs:
            a is the feature representation of the style image at that layer
            g is the feature representation of the generated image at that layer
        Output:
            the style loss at a certain layer (which is E_l in the paper)
        Hint: 1. you'll have to use the function _gram_matrix()
            2. we'll use the same coefficient for style loss as in the paper
            3. a and g are feature representation, not gram matrices
        """
        # M is the product of the first two dimensions of the feature map
        # N third dimension of the feature map
        N = a.shape[3]
        M = a.shape[1] * a.shape[2]
        A = self._gram_matrix(a, N=N, M=M)
        G = self._gram_matrix(g, N=N, M=M)
        coeff = 1 / (4 * (N ** 2) * (M ** 2))
        return coeff * tf.reduce_sum(tf.square(G - A))

    def _style_loss(self, A):
        """ Calculate the total style loss as a weighted sum
        of style losses at all style layers
        Hint: you'll have to use _single_style_loss()
        """
        n_layers = len(A)
        E = [self._single_style_loss(A[i], getattr(self.vgg, self.style_layers[i])) for i in range(n_layers)]
        self.style_loss = tf.reduce_sum([self.style_layer_w[i] * E[i] for i in range(n_layers)])

    def losses(self):
        with tf.variable_scope('losses'):
            with tf.Session() as sess:
                # assign content image to the input variable
                sess.run(self.input_img.assign(self.content_img))
                gen_img_content = getattr(self.vgg, self.content_layer)
                content_img_content = sess.run(gen_img_content)
            self._content_loss(content_img_content, gen_img_content)

            with tf.Session() as sess:
                sess.run(self.input_img.assign(self.style_img))
                style_layers = sess.run([getattr(self.vgg, layer) for layer in self.style_layers])
            self._style_loss(style_layers)

            self.total_loss = self.style_w * self.style_loss + self.content_w * self.content_loss

    def optimize(self):
        self.opt = tf.train.AdamOptimizer(self.lr).minimize(self.total_loss, global_step=self.gstep)

    def create_summary(self):
        with tf.name_scope('summary'):
            tf.summary.scalar('total_loss', self.total_loss)
            tf.summary.scalar('content_loss', self.content_loss)
            tf.summary.scalar('style_loss', self.style_loss)
            self.summary_op = tf.summary.merge_all()

    def build(self):
        self.create_input()
        self.load_vgg()
        self.losses()
        self.optimize()
        self.create_summary()

    def train(self, n_iters):
        skip_step = 1
        saver = tf.train.Saver()
        with tf.Session() as sess:

            sess.run(tf.global_variables_initializer())
            writer = tf.summary.FileWriter('graphs/transfer-style/lr' + str(self.lr), sess.graph)

            sess.run(self.input_img.assign(self.initial_img))
            ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/checkpoint'))
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)

            initial_step = self.gstep.eval()

            start_time = time.time()
            for index in range(initial_step, n_iters):
                if 5 <= index < 20:
                    skip_step = 10
                elif index >= 20:
                    skip_step = 20

                sess.run(self.opt)
                if (index + 1) % skip_step == 0:
                    gen_image, total_loss, summary = sess.run([self.input_img, self.total_loss, self.summary_op])

                    # add back the mean pixels we subtracted before
                    gen_image = gen_image + self.vgg.mean_pixels
                    writer.add_summary(summary, global_step=index)
                    print('Step {}\n   Sum: {:5.1f}'.format(index + 1, np.sum(gen_image)))
                    print('   Loss: {:5.1f}'.format(total_loss))
                    print('   Took: {} seconds'.format(time.time() - start_time))
                    start_time = time.time()

                    filename = 'drive/outputs/%d.png' % index
                    save_image(filename, gen_image)

                    if (index + 1) % 20 == 0:
                        saver.save(sess, 'checkpoints/style-transfer', global_step=index)


if __name__ == '__main__':
    setup()
    machine = StyleTransfer('drive/myself.jpg', 'drive/starrynight.jpg', 333, 250)
    machine.build()
    machine.train(1000)

VGG-19 pre-trained model is ready
INFO:tensorflow:Restoring parameters from checkpoints/style-transfer-599
Step 620
   Sum: 31297414.4
   Loss: 4506290.0
   Took: 5.692676544189453 seconds
Step 640
   Sum: 31278147.5
   Loss: 4451164.0
   Took: 4.223987817764282 seconds
Step 660
   Sum: 31259466.1
   Loss: 4398464.5
   Took: 5.4752585887908936 seconds
Step 680
   Sum: 31240661.8
   Loss: 4346468.0
   Took: 3.9004404544830322 seconds
Step 700
   Sum: 31221686.6
   Loss: 4294899.5
   Took: 4.532842397689819 seconds
Step 720
   Sum: 31202499.3
   Loss: 4243518.0
   Took: 4.796046733856201 seconds
Step 740
   Sum: 31183136.5
   Loss: 4192772.5
   Took: 4.985067367553711 seconds
Step 760
   Sum: 31163538.9
   Loss: 4142727.5
   Took: 4.026298761367798 seconds
Step 780
   Sum: 31143721.8
   Loss: 4093217.8
   Took: 4.055847883224487 seconds
Step 800
   Sum: 31123715.5
   Loss: 4044229.0
   Took: 7.791335105895996 seconds
Step 820
   Sum: 31103527.9
   Loss: 3995635.5
   Took: 4.4062077999114